In [2]:
import warnings
warnings.filterwarnings("ignore")

import os
import re

import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

from collections import defaultdict
from tqdm import tqdm
tqdm.pandas()

from sklearn.model_selection import train_test_split


# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("cahya/bert-base-indonesian-NER",  truncation=True)

In [3]:
MODEL_PRETRAIN = 'indobenchmark/indobert-base-p2'

In [4]:

from transformers import AutoTokenizer, BertTokenizer
tokenizer = BertTokenizer.from_pretrained(MODEL_PRETRAIN)

from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, BertForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(MODEL_PRETRAIN, num_labels=5)
model.train()

Downloading:   0%|          | 0.00/229k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [5]:
DEBUG = False

In [2]:
from torch.nn import functional as F
import torch
torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
print(device)

cpu


In [8]:
TARGET = 'POI/street'
POI = 'POI'
STREET = 'street'

SEED = 2021

In [9]:
df = pd.read_csv('C:/Users/Ale/Desktop/jupyter_folder/FYP/train.csv')
test = pd.read_csv('C:/Users/Ale/Desktop/jupyter_folder/FYP/test.csv')

In [10]:
def reconstruct_tokens(sent):
    sent = sent[1:-1]    
    sentences = []

    for idx, token in enumerate(sent):
        if token.startswith("##"):
            sentences[-1] += token.lstrip("##")
        else:
            sentences.append(token)
    return sentences


reconstruct_tokens(tokenizer.convert_ids_to_tokens(tokenizer(df['raw_address'].iloc[0])['input_ids']))
# tokenizer.convert_ids_to_tokens

['jl',
 'kapuk',
 'timur',
 'delta',
 'sili',
 'iii',
 'lippo',
 'cika',
 '11',
 'a',
 'cicau',
 'cikarang',
 'pusat']

In [11]:
if DEBUG:
    df = df.sample(10000)

In [12]:
df['clean_tokens'] = df['raw_address'].progress_apply(lambda x: reconstruct_tokens(tokenizer.convert_ids_to_tokens(tokenizer(x)['input_ids'])))

100%|██████████| 300000/300000 [01:48<00:00, 2767.74it/s]


In [13]:
test['clean_tokens'] = test['raw_address'].progress_apply(lambda x: reconstruct_tokens(tokenizer.convert_ids_to_tokens(tokenizer(x)['input_ids'])))

100%|██████████| 50000/50000 [00:18<00:00, 2772.84it/s]


In [14]:
df[POI] = df[TARGET].str.split('/', n = 2, expand = True)[0]
df[STREET] = df[TARGET].str.split('/', n = 2, expand = True)[1]

In [15]:
df[POI] = df[POI].progress_apply(lambda x: reconstruct_tokens(tokenizer.convert_ids_to_tokens(tokenizer(x)['input_ids'])))
df[STREET] = df[STREET].progress_apply(lambda x: reconstruct_tokens(tokenizer.convert_ids_to_tokens(tokenizer(x)['input_ids'])))

100%|██████████| 300000/300000 [00:50<00:00, 5934.60it/s]


In [16]:
import nltk
from nltk.metrics import distance

OTHER = 'O'

class LabelGenerator:
    
    def __init__(self):
        self.maping_poi = defaultdict(
            lambda: defaultdict(
                lambda: 0
            )
        )
        self.inv_maping_poi = defaultdict(
            lambda: defaultdict(
                lambda: 0
            )
        )

        self.maping_street = defaultdict(
            lambda: defaultdict(
                lambda: 0
            )
        )
        self.inv_maping_street = defaultdict(
            lambda: defaultdict(
                lambda: 0
            )
        )
    
    def calculate_score(self, token1, token2):
        scores = []
        for idx, v1 in enumerate(token1):
            v2 = token2[idx]
            ed = distance.edit_distance(v1, v2)
            scores.append(ed)
        return np.mean(scores)
            
    def generate_score(self, tokens, target, debug=False):
        scores = {
            'idx':[],
            'scores': []
        }
        if debug:
            print(tokens)
            print(target)
        L_target = len(target)
        L_tokens = len(tokens)
        
        for i in range(0, L_tokens - L_target + 1):
            if debug:
                print(i)
            scores['idx'].append(set(range(i,i+L_target)))
            scores['scores'].append(self.calculate_score(tokens[i:i+L_target], target))
        return pd.DataFrame(scores)

    def generate_single_label(self, address_token, street_token, POI_token, debug=False):
        
        labels = [OTHER for _ in address_token]
    
        curstreet = 0
        curpoi = 0
        L_addrs = len(address_token)
        L_street = len(street_token)
        L_poi = len(POI_token)
        
        if L_street == 0 and L_poi == 0:
            return labels

#         if L_street == 0 or L_poi == 0:
#             return old_generate_label(address_token, street_token, POI_token, debug=False)
            
        street_score = self.generate_score(address_token, street_token, debug)
        POI_score = self.generate_score(address_token, POI_token, debug)
                
        street_score = street_score.sort_values('scores').reset_index(drop=True)
        POI_score = POI_score.sort_values('scores').reset_index(drop=True)
        
        poi_idx = 0
        street_idx = 0
        if debug:
            print(L_addrs, L_street, L_poi)
            print('street', street_score.shape)
            print(street_score)
            print('poi', POI_score.shape)
            print(POI_score)            
        
        
        street_score_shape = street_score.shape[0]
        poi_score_shape = POI_score.shape[0]
        
        
        if L_street == 0:
            street_idx = -1
            street_score_shape = 0
        
        if L_poi == 0:
            poi_idx = -1
            poi_score_shape = 0
        
        
        while poi_idx < poi_score_shape and street_idx < street_score_shape:
            
            if poi_score_shape == 0:
                poi_labels = set()
            else:
                poi_labels = POI_score['idx'].iloc[poi_idx]
                
            if street_score_shape == 0:
                street_labels = set()
            else:
                street_labels = street_score['idx'].iloc[street_idx]

            if debug:
                print(street_score_shape, street_labels, poi_score_shape,poi_labels,  len(poi_labels.intersection(street_labels)))
            
            if len( poi_labels.intersection(street_labels)) == 0:
                if debug:
                    print('masuk')
                list_poi_labels = list(sorted(poi_labels))
                list_street_labels = list(sorted(street_labels))

                current = None
                
                cur_poi_idx = 0
                cur_street_idx = 0

                for idx, _ in enumerate(labels):
                    if idx in poi_labels:
                        

                        self.maping_poi[address_token[idx]][POI_token[cur_poi_idx]] += 1
                        self.inv_maping_poi[POI_token[cur_poi_idx]][address_token[idx]] += 1                           
                        cur_poi_idx += 1
                        
                        if current != 'POI':
                            current = 'POI'
                            labels[idx] = 'B-POI'
                        else:
                            labels[idx] = 'I-POI'
                    elif idx in street_labels:
                        self.maping_street[address_token[idx]][street_token[cur_street_idx]] += 1
                        self.inv_maping_street[street_token[cur_street_idx]][address_token[idx]] += 1                           
                        cur_street_idx += 1
                        
                        
                        if current != 'STREET':
                            current = 'STREET'
                            labels[idx] = 'B-STREET'
                        else:
                            labels[idx] = 'I-STREET'
                    else:
                        current = None
                return labels
            if len(poi_labels) < len(street_labels):
                poi_idx += 1
            else:
                street_idx += 1
        
        return []

In [17]:
label_gen = LabelGenerator()

In [18]:
labels = df[['clean_tokens', STREET, POI]].progress_apply(lambda x: label_gen.generate_single_label(*x), axis=1)

100%|██████████| 300000/300000 [13:22<00:00, 373.85it/s]


In [19]:
df['labels'] = labels

In [20]:
train, val = train_test_split(df, shuffle=True, random_state=SEED)

In [21]:
train['split'] = "train"
val['split'] = "val"

In [22]:
label_list = [OTHER, 'B-STREET', 'I-STREET', 'B-POI', 'I-POI']
label_tags = {v: i for i, v in enumerate(label_list)}

In [23]:
len_labels = df['labels'].apply(len)
len_token = df['clean_tokens'].apply(len)

In [24]:
df[len_token != len_labels]

id                                        raw_address  \
4328      4328                                   tk tahfidz baby,   
4835      4835        quint cv dukuh setro iv a gading tambaksari   
6563      6563                  pt. adira dinamika multi fina tbk   
6804      6804                                       kuburan kedo   
7543      7543                                      komp tamalate   
...        ...                                                ...   
291697  291697                              gram cilacap sidareja   
291762  291762  raya wonos toko alat- listrik mirah baleharjo ...   
294785  294785                   toko atika lintas sumatera kemu,   
296099  296099                        clan, i gusti ngurah rai 39   
297771  297771                        depot air minum, lin sumate   

                                         POI/street  \
4328                            tk tahfidz baby-qu/   
4835                  quintrix, cv/dukuh setro iv a   
6563         pt. adira dinamika multi finance, tbk/   
6804                               kuburan kedo'an/   
7543                                       /komp.ha   
...                                             ...   
291697                      gramedia cilacap-90101/   
291762  toko alat-alat listrik mirah/raya wonosari-   
294785        toko atika/lintas sumatera-bukit kemu   
296099                  clandy's/i gusti ngurah rai   
297771               depot air minum/lin sumatera-k   

                                             clean_tokens  \
4328                               [tk, tahfidz, baby, ,]   
4835    [quint, cv, dukuh, setro, iv, a, gading, tamba...   
6563           [pt, ., adira, dinamika, multi, fina, tbk]   
6804                                      [kuburan, kedo]   
7543                                     [komp, tamalate]   
...                                                   ...   
291697                          [gram, cilacap, sidareja]   
291762  [raya, wonos, toko, alat, -, listrik, mirah, b...   
294785           [toko, atika, lintas, sumatera, kemu, ,]   
296099               [clan, ,, i, gusti, ngurah, rai, 39]   
297771                [depot, air, minum, ,, lin, sumate]   

                                                     POI  \
4328                          [tk, tahfidz, baby, -, qu]   
4835                                   [quintrix, ,, cv]   
6563    [pt, ., adira, dinamika, multi, finance, ,, tbk]   
6804                              [kuburan, kedo, ', an]   
7543                                                  []   
...                                                  ...   
291697                     [gramedia, cilacap, -, 90101]   
291762             [toko, alat, -, alat, listrik, mirah]   
294785                                     [toko, atika]   
296099                                    [clandy, ', s]   
297771                               [depot, air, minum]   

                                    street labels  
4328                                    []     []  
4835                 [dukuh, setro, iv, a]     []  
6563                                    []     []  
6804                                    []     []  
7543                         [komp, ., ha]     []  
...                                    ...    ...  
291697                                  []     []  
291762                 [raya, wonosari, -]     []  
294785  [lintas, sumatera, -, bukit, kemu]     []  
296099             [i, gusti, ngurah, rai]     []  
297771               [lin, sumatera, -, k]     []  

[346 rows x 7 columns]

In [25]:
df = df[len_token == len_labels]

In [26]:
train, val = train_test_split(df, shuffle=True, random_state=SEED)

In [27]:
train_dataset_list = {
    'sentences':[],
    'ner':[],
}
for idx, v  in tqdm(train.iterrows(), total=train.shape[0]):
    train_dataset_list['sentences'].append(v['clean_tokens'])
    train_dataset_list['ner'].append([label_tags[x] for x in v['labels']])
    assert len(train_dataset_list['ner'][-1]) == len(train_dataset_list['sentences'][-1])

100%|██████████| 224740/224740 [00:18<00:00, 12048.13it/s]


In [28]:
val_dataset_list = {
    'sentences':[],
    'ner':[]
}
for idx, v  in tqdm(val.iterrows(), total=val.shape[0]):
    val_dataset_list['sentences'].append(v['clean_tokens'])
    val_dataset_list['ner'].append([label_tags[x] for x in v['labels']])
    assert len(val_dataset_list['ner'][-1]) == len(val_dataset_list['sentences'][-1])

100%|██████████| 74914/74914 [00:06<00:00, 12030.57it/s]


In [29]:
!pip3 install -q datasets seqeval

In [30]:
from datasets import Dataset

In [31]:
val_dataset = Dataset.from_dict({k:val_dataset_list[k] for k in ['sentences', 'ner']})

In [32]:
train_dataset = Dataset.from_dict({k:train_dataset_list[k] for k in ['sentences', 'ner']})

In [33]:
# tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

In [34]:
# from transformers import DataCollatorForTokenClassification

# data_collator = DataCollatorForTokenClassification(tokenizer)

In [35]:
from datasets import load_metric
metric = load_metric("seqeval")


Downloading:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

In [36]:
# tokenizer

In [37]:
special_tokens = {'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}
special_tokens = {special_tokens[k] for k in special_tokens}

In [38]:
def tokenize_and_align_labels1(examples):
    tokenized_inputs = tokenizer(examples["sentences"], truncation=True, padding=True, is_split_into_words=True, return_tensors="pt")

    labels = []
    for i, label in enumerate(examples["ner"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [39]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["sentences"], truncation=True, padding=True, is_split_into_words=True, return_tensors="pt")

#     outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
    labels = []
    for i, label in enumerate(examples["ner"]):
        
        _labels = []
        neridx = 0
        tokens = tokenizer.convert_ids_to_tokens(tokenized_inputs["input_ids"][i])
#         print(tokens)
#         print(label)
        for idx, tok in enumerate(tokens):
            if tok in special_tokens:
                _labels.append(-100)
            elif tok.startswith("##"):
                _labels.append(_labels[-1])
            else:
                try:
                    _labels.append(label[neridx])
                except:
                    print('-'*10)
                    print(neridx, idx)
                    print(label)
                    print(tokens)
                    print(tok)
                    print('-'*10)
                    raise
                neridx += 1
        
        labels.append(_labels)
        
    tokenized_inputs["labels"] = torch.tensor(labels)
    return tokenized_inputs

# tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])

In [40]:
def compute_metrics(predictions, labels):
    predictions = torch.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [41]:
batchsize = 32

enumerator = list(range(0, train_dataset.num_rows, batchsize))
tot = 0

train_data = [tokenize_and_align_labels(train_dataset[i:i+batchsize])  for i in enumerator]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [42]:
batchsize = 32

enumerator = list(range(0, val_dataset.num_rows, batchsize))
tot = 0

val_data = [tokenize_and_align_labels(val_dataset[i:i+batchsize])  for i in enumerator]

In [43]:
class RunningAverage():
    """A simple class that maintains the running average of a quantity
    Example:
    ```
    loss_avg = RunningAverage()
    loss_avg.update(2)
    loss_avg.update(4)
    loss_avg() = 3
    ```
    """

    def __init__(self):
        self.steps = 0
        self.total = 0

    def update(self, val):
        self.total += val
        self.steps += 1

    def __call__(self):
        return self.total / float(self.steps)

In [44]:
model = model.to(device)

In [45]:
if DEBUG:
    BREAKPOINT = 1
else:
    BREAKPOINT = 20
TIME_STOP = 4 * 3600

In [47]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=1e-5)

max_f1 = 0
last_update = 0
last_best = None
for epoch in range(50):
    if time.time() - START_TIME > TIME_STOP:
        break
    print()
    print('epoch', epoch)    
    for idx, train_param in tqdm(enumerate(train_data), total=len(train_data)):
        optimizer.zero_grad()
        outputs = model(**{k: train_param[k].to(device) for k in train_param.keys()})
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    metrics = {
        'precision': RunningAverage(),
        'recall': RunningAverage(),
        'f1': RunningAverage(),
        'accuracy': RunningAverage()
    }
    for idx, val_param in enumerate(val_data):
        outputs = model(**{k: val_param[k].to(device) for k in val_param.keys()})
        softmax = F.softmax(outputs.logits, dim = -1)
        _m = compute_metrics(softmax, val_param.labels)

        for k in _m:
            metrics[k].update(_m[k])
    f1 = metrics['f1']()
    if f1 > max_f1:
        max_f1 = f1
        save_name = 'pretrained_{}-{:0.7f}/'.format(epoch, f1)
        last_best = save_name
        print('breakpoint, save to', save_name)
        model.save_pretrained(save_name)
    print("\n".join(["{}:{}{:0.10f}".format(k," "*(10-len(k)), metrics[k]()) for k in metrics]))

  0%|          | 0/7024 [00:00<?, ?it/s]


epoch 0


100%|██████████| 7024/7024 [12:24<00:00,  9.44it/s]


breakpoint, save to pretrained_0-0.8214893/


  0%|          | 1/7024 [00:00<15:28,  7.56it/s]

...

precision: 0.7935009780
recall:    0.8510016634
f1:        0.8195739189
accuracy:  0.9102240282

epoch 13


  0%|          | 1/7024 [00:00<17:26,  6.71it/s]

precision: 0.7921497533
recall:    0.8486871351
f1:        0.8178314841
accuracy:  0.9102071138

epoch 14


100%|██████████| 7024/7024 [12:16<00:00,  9.54it/s]


precision: 0.7924989702
recall:    0.8516101444
f1:        0.8193728699
accuracy:  0.9117248935


In [48]:
model.save_pretrained('finale/')

In [49]:
model = model.eval()

In [50]:
try:
    print(last_best)
    model = AutoModelForTokenClassification.from_pretrained(last_best).to(device)
except Exception  as e:
    print('err', str(e))

pretrained_4-0.8417658/


In [51]:
# with pytorch.no_grad():
#     torch.cuda.empty_cache()

In [52]:
torch.cuda.empty_cache()

In [53]:
train_predicts = []
for idx, _data in tqdm(enumerate(train_data), total=len(train_data)):
    outputs = model(**{k: _data[k].to(device) for k in _data.keys()})
    softmax = F.softmax(outputs.logits, dim = -1).cpu().detach().numpy().astype(np.float16)
    train_predicts.append(softmax)

100%|██████████| 7024/7024 [03:20<00:00, 35.06it/s]


In [54]:
import pickle
with open('train_predicts.pkl', 'wb') as f:
    pickle.dump(train_predicts, f)

In [55]:
import pickle
with open('train_data.pkl', 'wb') as f:
    pickle.dump(train_data, f)

In [56]:
train.to_parquet('train.pqt')

In [57]:
val_predicts = []
for idx, _data in tqdm(enumerate(val_data), total=len(val_data)):
    outputs = model(**{k: _data[k].to(device) for k in _data.keys()})
    softmax = F.softmax(outputs.logits, dim = -1).cpu().detach().numpy().astype(np.float16)
    val_predicts.append(softmax)

100%|██████████| 2342/2342 [01:07<00:00, 34.94it/s]


In [58]:
import pickle
with open('val_predicts.pkl', 'wb') as f:
    pickle.dump(val_predicts, f)

In [59]:
import pickle
with open('val_data.pkl', 'wb') as f:
    pickle.dump(val_data, f)

In [60]:
val.to_parquet('val.pqt')

In [61]:
test_dataset_list = {
    'sentences':[]
}
for idx, v  in tqdm(test.iterrows(), total=test.shape[0]):
    test_dataset_list['sentences'].append(v['clean_tokens'])
# test_tokenized = tokenizer(test_dataset_list["sentences"], truncation=True, padding=True, is_split_into_words=True, return_tensors="pt")

100%|██████████| 50000/50000 [00:03<00:00, 14222.63it/s]


In [62]:
batchsize = 16

enumerator = list(range(0, len(test_dataset_list['sentences']), batchsize))
tot = 0

test_data = [
    tokenizer(test_dataset_list['sentences'][i:i+batchsize], truncation=True, padding=True, is_split_into_words=True, return_tensors="pt")
    for i in enumerator
]

In [63]:
test_predicts = []
for idx, _data in tqdm(enumerate(test_data), total=len(test_data)):
    outputs = model(**{k: _data[k].to(device) for k in _data.keys()})
    softmax = F.softmax(outputs.logits, dim = -1).cpu().detach().numpy().astype(np.float16)
    test_predicts.append(softmax)

100%|██████████| 3125/3125 [00:47<00:00, 65.31it/s]


In [64]:
import pickle
with open('test_predicts.pkl', 'wb') as f:
    pickle.dump(test_predicts, f)

In [65]:
import pickle
with open('test_data.pkl', 'wb') as f:
    pickle.dump(test_data, f)

In [66]:
test.to_parquet('test.pqt')